In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import deque
from surprise import Dataset, NormalPredictor, Reader
from surprise.model_selection import cross_validate
import os
from sklearn.model_selection import train_test_split
import pickle
from surprise import accuracy
from surprise import accuracy, Dataset, Reader, SVD, KNNBaseline
from surprise.model_selection import PredefinedKFold
import pprint
import tempfile
from typing import Dict, Text
import tensorflow as tf
import tensorflow_recommenders as tfrs

In [2]:
#!pwd

/Users/alecclarkfeather/capstone_project


In [3]:
movie_titles = pd.read_csv('movie_titles.csv', header=None, encoding='ISO-8859-1', 
                          names=['movie_id', 'year', 'title', 'description1', 'description2',
                                'description3'])

In [4]:
movie_titles = movie_titles.drop(columns=['description1', 'description2', 'description3'], axis=1)

In [5]:
RMSE_PATH = '/Users/alecclarkfeather/capstone_project/rmse_train_test_folds/'

df_rmse_train0 = pd.read_csv(RMSE_PATH + 'train_df0.csv')
df_rmse_test0 = pd.read_csv(RMSE_PATH + 'test_df0.csv')

In [6]:
#df_rmse_train0.head(10)

,Unnamed: 0,user_id,rating,date,movie_id
0,4544200,1590329,3.0,2004-06-24,7627
1,1880455,576570,4.0,2005-01-26,14606
2,3723821,1757599,3.0,2005-06-03,9156
3,2798566,1048852,4.0,2003-12-02,15107
4,4760187,2187348,3.0,2005-07-18,14644
5,1788227,1029245,3.0,2005-10-10,17169
6,633589,1090134,4.0,2005-05-08,7620
7,3071715,529406,2.0,2005-01-11,15582
8,1933360,138807,1.0,2005-10-20,4979
9,1840081,1096013,4.0,2005-08-23,13763


In [7]:
#df_rmse_test0.head(10)

,Unnamed: 0,user_id,rating,date,movie_id
0,642740,573762,4.0,2003-08-26,9438
1,2737204,1560019,4.0,2004-02-03,5760
2,4489424,2120159,1.0,2005-10-04,1542
3,2771061,1388182,1.0,2005-03-12,12034
4,2590429,142077,3.0,2005-10-10,15755
5,1877955,2139511,4.0,2005-10-10,4432
6,2821818,567989,4.0,2005-07-06,15758
7,4095949,2231354,3.0,2005-01-25,16668
8,952879,975377,4.0,2005-09-08,4432
9,2859300,1441072,4.0,2004-05-31,14218


In [8]:
df_rmse_train0 = df_rmse_train0.drop('Unnamed: 0', axis=1)
df_rmse_test0 = df_rmse_test0.drop('Unnamed: 0', axis=1)

In [9]:
#df_rmse_train0.head()

,user_id,rating,date,movie_id
0,1590329,3.0,2004-06-24,7627
1,576570,4.0,2005-01-26,14606
2,1757599,3.0,2005-06-03,9156
3,1048852,4.0,2003-12-02,15107
4,2187348,3.0,2005-07-18,14644


In [10]:
#df_rmse_test0.head()

,user_id,rating,date,movie_id
0,573762,4.0,2003-08-26,9438
1,1560019,4.0,2004-02-03,5760
2,2120159,1.0,2005-10-04,1542
3,1388182,1.0,2005-03-12,12034
4,142077,3.0,2005-10-10,15755


In [12]:
df_rmse_train0 = df_rmse_train0[['user_id', 'movie_id', 'rating']]
df_rmse_test0 = df_rmse_test0[['user_id', 'movie_id', 'rating']]

In [13]:
#df_rmse_train0.head()

,user_id,movie_id,rating
0,1590329,7627,3.0
1,576570,14606,4.0
2,1757599,9156,3.0
3,1048852,15107,4.0
4,2187348,14644,3.0


In [14]:
#df_rmse_test0.head()

,user_id,movie_id,rating
0,573762,9438,4.0
1,1560019,5760,4.0
2,2120159,1542,1.0
3,1388182,12034,1.0
4,142077,15755,3.0


In [21]:
#df_rmse_train0['rating'] = df_rmse_train0['rating'].astype(float)
#df_rmse_test0['rating'] = df_rmse_test0['rating'].astype(float)

In [18]:
#reader = Reader(rating_scale=(1,5))
#train_rmse0 = Dataset.load_from_df(df_rmse_train0, reader)
#test_rmse0 = Dataset.load_from_df(df_rmse_test0, reader)

In [20]:
#baseline_algo_1a = NormalPredictor()

#baseline_algo_1a.fit(train_rmse0)
#preds_1a = baseline_algo1a.test(test_rmse0)

#baseline_algo_1b = NormalPredictor()

#baseline_algo_1b.fit(test_rmse0)
#preds_1b = baseline_algo_1b.test(train_rmse0)

#rmse_1a = accuracy.rmse(preds_1a)
#rmse_1b = accuracy.rmse(preds_1b)

#print(rmse_1a)
#print(rmse_1b)

In [22]:
#!pwd

/Users/alecclarkfeather/capstone_project


In [27]:
PATH = '/Users/alecclarkfeather/capstone_project/'

os.mkdir(PATH + 'rmse_folds_actual')

In [28]:
RMSE_PATH_ACTUAL = PATH + 'rmse_folds_actual/'
df_rmse_train0.to_csv(RMSE_PATH_ACTUAL + 'actual_rmse_train0.csv', index=False, header=None)
df_rmse_test0.to_csv(RMSE_PATH_ACTUAL + 'actual_rmse_test0.csv', index=False, header=None)

In [30]:
reader = Reader(sep=',',rating_scale=(1,5))

train_file = RMSE_PATH_ACTUAL + 'actual_rmse_train0.csv'
test_file = RMSE_PATH_ACTUAL + 'actual_rmse_test0.csv'
folds_files = [(train_file, test_file)]

data = Dataset.load_from_folds(folds_files, reader=reader)

pkf = PredefinedKFold()

baseline_algo = NormalPredictor()

for trainset, testset in pkf.split(data):
    baseline_algo.fit(trainset)
    preds = baseline_algo.test(testset)
    accuracy.rmse(preds, verbose=True)

RMSE: 1.4778


In [31]:
reader = Reader(sep=',',rating_scale=(1,5))

test_file = RMSE_PATH_ACTUAL + 'actual_rmse_train0.csv'
train_file = RMSE_PATH_ACTUAL + 'actual_rmse_test0.csv'
folds_files = [(train_file, test_file)]

data = Dataset.load_from_folds(folds_files, reader=reader)

pkf = PredefinedKFold()

baseline_algo = NormalPredictor()

for trainset, testset in pkf.split(data):
    baseline_algo.fit(trainset)
    preds = baseline_algo.test(testset)
    accuracy.rmse(preds, verbose=True)

RMSE: 1.4775


In [33]:
reader = Reader(sep=',',rating_scale=(1,5))

train_file = RMSE_PATH_ACTUAL + 'actual_rmse_train0.csv'
test_file = RMSE_PATH_ACTUAL + 'actual_rmse_test0.csv'
folds_files = [(train_file, test_file)]

data = Dataset.load_from_folds(folds_files, reader=reader)

pkf = PredefinedKFold()

sim_options = {"name": "pearson_baseline", "user_based": False}
baseline_algo = KNNBaseline(sim_options=sim_options)

for trainset, testset in pkf.split(data):
    baseline_algo.fit(trainset)
    preds = baseline_algo.test(testset)
    accuracy.rmse(preds, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.0669


In [34]:
reader = Reader(sep=',',rating_scale=(1,5))

test_file = RMSE_PATH_ACTUAL + 'actual_rmse_train0.csv'
train_file = RMSE_PATH_ACTUAL + 'actual_rmse_test0.csv'
folds_files = [(train_file, test_file)]

data = Dataset.load_from_folds(folds_files, reader=reader)

pkf = PredefinedKFold()

sim_options = {"name": "pearson_baseline", "user_based": False}
baseline_algo = KNNBaseline(sim_options=sim_options)

for trainset, testset in pkf.split(data):
    baseline_algo.fit(trainset)
    preds = baseline_algo.test(testset)
    accuracy.rmse(preds, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.0674


In [35]:
reader = Reader(sep=',',rating_scale=(1,5))

train_file = RMSE_PATH_ACTUAL + 'actual_rmse_train0.csv'
test_file = RMSE_PATH_ACTUAL + 'actual_rmse_test0.csv'
folds_files = [(train_file, test_file)]

data = Dataset.load_from_folds(folds_files, reader=reader)

pkf = PredefinedKFold()

#sim_options = {"name": "pearson_baseline", "user_based": False}
baseline_algo = SVD()

for trainset, testset in pkf.split(data):
    baseline_algo.fit(trainset)
    preds = baseline_algo.test(testset)
    accuracy.rmse(preds, verbose=True)

RMSE: 1.0063


In [36]:
reader = Reader(sep=',',rating_scale=(1,5))

test_file = RMSE_PATH_ACTUAL + 'actual_rmse_train0.csv'
train_file = RMSE_PATH_ACTUAL + 'actual_rmse_test0.csv'
folds_files = [(train_file, test_file)]

data = Dataset.load_from_folds(folds_files, reader=reader)

pkf = PredefinedKFold()

#sim_options = {"name": "pearson_baseline", "user_based": False}
baseline_algo = SVD()

for trainset, testset in pkf.split(data):
    baseline_algo.fit(trainset)
    preds = baseline_algo.test(testset)
    accuracy.rmse(preds, verbose=True)

RMSE: 1.0065


In [37]:
reader = Reader(sep=',',rating_scale=(1,5))

test_file = RMSE_PATH_ACTUAL + 'actual_rmse_train0.csv'
train_file = RMSE_PATH_ACTUAL + 'actual_rmse_test0.csv'
folds_files = [(train_file, test_file)]

data = Dataset.load_from_folds(folds_files, reader=reader)

pkf = PredefinedKFold()

#sim_options = {"name": "pearson_baseline", "user_based": False}
baseline_algo = SVD()
scores = []

for trainset, testset in pkf.split(data):
    baseline_algo.fit(trainset)
    preds = baseline_algo.test(testset)
    score = accuracy.rmse(preds, verbose=True)
    scores.append(score)

RMSE: 1.0068


In [38]:
#scores

[1.0068391121351412]

In [44]:
train_rmse0 = pd.read_csv(RMSE_PATH_ACTUAL + 'actual_rmse_train0.csv', header=None)
test_rmse0 = pd.read_csv(RMSE_PATH_ACTUAL + 'actual_rmse_test0.csv', header=None)

In [45]:
#train_rmse0.head()

,0,1,2
0,1590329,7627,3.0
1,576570,14606,4.0
2,1757599,9156,3.0
3,1048852,15107,4.0
4,2187348,14644,3.0


In [46]:
#test_rmse0.head()

,0,1,2
0,573762,9438,4.0
1,1560019,5760,4.0
2,2120159,1542,1.0
3,1388182,12034,1.0
4,142077,15755,3.0


In [47]:
train_rmse0.columns = ['user_id', 'movie_id', 'rating']
test_rmse0.columns = ['user_id', 'movie_id', 'rating']

In [50]:
train_movie_ids = list(train_rmse0['movie_id'].unique())
test_movie_ids = list(test_rmse0['movie_id'].unique())

#print(set(train_movie_ids)==set(test_movie_ids))

In [51]:
#train_movie_ids

In [53]:
#set(train_movie_ids).difference(set(test_movie_ids))

In [54]:
#all_movie_ids = set(train_movie_ids).union(set(test_movie_ids))

In [55]:
#all_movie_ids = np.array(all_movie_ids)

In [56]:
train_user_ids = list(train_rmse0['user_id'].unique())
test_user_ids = list(test_rmse0['user_id'].unique())

In [57]:
#print(set(train_user_ids)==set(test_user_ids))

False


In [61]:
#len(set(train_user_ids).difference(set(test_user_ids)))

883

In [62]:
#all_user_ids = set(train_user_ids).union(set(test_user_ids))

In [63]:
#all_user_ids = np.array(all_user_ids)

In [66]:
#all_user_ids

In [70]:
#all_user_ids = list(all_user_ids)
#all_user_ids

In [75]:
for movie in test_movie_ids:
    if movie not in train_movie_ids:
        train_movie_ids.append(movie)

In [76]:
for user in test_user_ids:
    if user not in train_user_ids:
        train_user_ids.append(user)

In [78]:
#len(train_movie_ids)

15639

In [162]:
class DeepRecRmseModel(tf.keras.Model):
    
    def __init__(self):
        super().__init__()
        embedding_dimension = 16
        
        self.user_embeddings = tf.keras.Sequential([
            tf.keras.layers.IntegerLookup(vocabulary=train_user_ids, mask_token=None),
            tf.keras.layers.Embedding(len(train_user_ids)+1, embedding_dimension)
        ])
        
        self.movie_embeddings = tf.keras.Sequential([
            tf.keras.layers.IntegerLookup(vocabulary=train_movie_ids, mask_token=None),
            tf.keras.layers.Embedding(len(train_movie_ids)+1, embedding_dimension)
        ])
        
        self.ratings = tf.keras.Sequential([
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(1)
        ])
        
    def call(self, inputs):
        user_id, movie_id = inputs
        user_embedding = self.user_embeddings(user_id)
        movie_embedding = self.movie_embeddings(movie_id)
        return self.ratings(tf.concat([tf.reshape(user_embedding, shape=[-1,1]), 
                                       tf.reshape(movie_embedding, shape=[-1,1])], axis=1))

In [ ]:
#task = tfrs.tasks.Ranking(loss=tf.keras.losses.MeanSquaredError(),
                         #metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [163]:
class NetflixRmseModel(tfrs.models.Model):
    
    def __init__(self):
        super().__init__()
        self.rmse_model: tf.keras.Model = DeepRecRmseModel()
        self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(loss=tf.keras.losses.MeanSquaredError(),
                                                             metrics=[tf.keras.metrics.RootMeanSquaredError()])
        
    def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
        return self.rmse_model((features['user_id'], features['movie_id']))
    
    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        labels = features.pop('rating')
        predicted_ratings = self(features)
        return self.task(labels=labels, predictions=predicted_ratings)
    

In [164]:
deep_baseline_model = NetflixRmseModel()
deep_baseline_model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.3))

In [87]:
#train_rmse0_tf = train_rmse0.map(lambda x: {
    #'user_id': x['user_id'],
   # 'movie_id': x['movie_id'],
    #'rating': x['rating']
#})

In [86]:
#history_baseline = deep_baseline_model.fit(tf.convert_to_tensor(train_rmse0.values), epochs=3, verbose=2)

In [165]:
train_rmse0_tf = tf.data.Dataset.from_tensor_slices(dict(train_rmse0))

In [166]:
for x in train_rmse0_tf.take(5):
    print(x)

{'user_id': <tf.Tensor: shape=(), dtype=int64, numpy=1590329>, 'movie_id': <tf.Tensor: shape=(), dtype=int64, numpy=7627>, 'rating': <tf.Tensor: shape=(), dtype=float64, numpy=3.0>}
{'user_id': <tf.Tensor: shape=(), dtype=int64, numpy=576570>, 'movie_id': <tf.Tensor: shape=(), dtype=int64, numpy=14606>, 'rating': <tf.Tensor: shape=(), dtype=float64, numpy=4.0>}
{'user_id': <tf.Tensor: shape=(), dtype=int64, numpy=1757599>, 'movie_id': <tf.Tensor: shape=(), dtype=int64, numpy=9156>, 'rating': <tf.Tensor: shape=(), dtype=float64, numpy=3.0>}
{'user_id': <tf.Tensor: shape=(), dtype=int64, numpy=1048852>, 'movie_id': <tf.Tensor: shape=(), dtype=int64, numpy=15107>, 'rating': <tf.Tensor: shape=(), dtype=float64, numpy=4.0>}
{'user_id': <tf.Tensor: shape=(), dtype=int64, numpy=2187348>, 'movie_id': <tf.Tensor: shape=(), dtype=int64, numpy=14644>, 'rating': <tf.Tensor: shape=(), dtype=float64, numpy=3.0>}


In [167]:
train_rmse0_tf = train_rmse0_tf.map(lambda x: {'user_id': x['user_id'],
    'movie_id': x['movie_id'],
    'rating': x['rating']
})

In [152]:
#train_rmse0_tf_cache = train_rmse0_tf.cache()

In [169]:
#history_baseline = deep_baseline_model.fit(train_rmse0_tf, epochs=1, batch_size=20000)